In [2]:
# Import dependendencies
import gmaps
import pandas as pd
import requests

#Import API key
from config import g_key

In [5]:
# Read and store CSV file as df
cityDataDF = pd.read_csv("weather_data/cities.csv")
cityDataDF.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use API key
gmaps.configure(api_key=g_key)

In [14]:
# Heatmap of temperature
locations = cityDataDF[["Lat","Lng"]]
maxTemp = cityDataDF["Max Temp"]

fig = gmaps.figure(center = (30.0,31.0), zoom_level=1.1)
heat_layer = gmaps.heatmap_layer(locations, weights = [max(0,temp) for temp in maxTemp], dissipating= False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of humidity
locations = cityDataDF[["Lat","Lng"]]
humidity = cityDataDF["Humidity"]

fig = gmaps.figure(center = (30.0,31.0), zoom_level=1.1)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating= False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Ask customer for max and min preferred temperature
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [40]:
# Filter dataset to find cities that meet the criteria
preferred_cities_df = cityDataDF.loc[(cityDataDF["Max Temp"] >= min_temp) & (cityDataDF["Max Temp"] <= max_temp)]
preferred_cities_df.count()

City_ID       123
City          123
Country       123
Date          123
Lat           123
Lng           123
Max Temp      123
Humidity      123
Cloudiness    123
Wind Speed    123
dtype: int64

In [41]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Laguna,US,98.44,38.4210,-121.4238,
2,Nanakuli,US,83.03,21.3906,-158.1547,
5,Kapaa,US,82.72,22.0752,-159.3190,
6,Porbandar,IN,80.46,21.6422,69.6093,
10,Hamilton,US,81.88,39.1834,-84.5333,
16,Dhidhdhoo,MV,81.99,6.8833,73.1000,
21,Portland,US,85.10,45.5234,-122.6762,
26,Hithadhoo,MV,82.49,-0.6000,73.0833,
31,Riyadh,SA,93.34,24.6877,46.7219,
32,Amapa,BR,85.62,1.0000,-52.0000,


In [42]:
# Set params to search for hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": ""
}

# Iterate through hotel_df to find hotels
for index,row in hotel_df.iterrows():
    # Get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")


Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [46]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))